*For the final submission, the hyperparameter tuning of the best performing models (which were determined in initial training) is done with GridSearchCV, which fits all possible candidates in a search space. Thus giving the most optimal parameters. The model are then trained with these parameters on the entire dataset for each of the camera id* \
For training the models, the same apporach explained in `train.ipynb` is used here.

### Importing Dependencies

In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Tuple

from sklearn.preprocessing import StandardScaler
from joblib import dump, load
from sklearn.model_selection import train_test_split

from cuml.linear_model import LinearRegression, Lasso, ElasticNet
from cuml.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from joblib import parallel_backend
from ray.util.joblib import register_ray
register_ray()
import ray
from cuml.common.device_selection import set_global_device_type

np.random.seed(42)

### Connect to the ray cluster

In [2]:
set_global_device_type("GPU")
ray.init("auto")

2024-08-25 10:55:37,428	INFO worker.py:1596 -- Connecting to existing Ray cluster at address: 10.19.3.211:6379...
2024-08-25 10:55:37,444	INFO worker.py:1772 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.12
Ray version:,2.34.0
Dashboard:,http://127.0.0.1:8265


In [3]:
dataset_dir = Path(r"/home/user1/codes/bangaluru_mobility_codes/submission/future_counts/data")
save_dir = Path(r"/home/user1/codes/bangaluru_mobility_codes/submission/future_counts/model_training/results_entire_dataset_each_cam_id")

In [4]:
acc_models = {"RandomForest","LinearRegression","Lasso","ElasticNet","XGBRegressor","CatBoostingRegressor"}
target_column = "count"

In [5]:
# Specify parameters and distributions to sample from for hyper-parameter tuning

params={
        "RandomForest":{
            'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
            'max_features':['sqrt','log2'],
            'n_estimators': [8,16,32,64,128,256,512,1024],
            'bootstrap': [False, True]
        },
        "GradientBoosting":{
            'loss':['squared_error', 'huber', 'absolute_error', 'quantile'],
            'learning_rate':[.1,.01,.05,.001],
            'subsample':[0.6,0.7,0.75,0.8,0.85,0.9],
            'criterion':['squared_error', 'friedman_mse'],
            'max_features':['sqrt','log2'],
            'n_estimators': [8,16,32,64,128,256]
        },
        "LinearRegression":{
            'algorithm': ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi']
        },
        "XGBRegressor":{
            'learning_rate': np.linspace(0.001,0.2,8),
            'n_estimators': np.arange(8,512,50),
            'gamma': np.linspace(0,5,10),
            'reg_alpha': np.logspace(-3,1,10),
            'reg_lambda': np.logspace(-3,1,10)
        },
        "CatBoostingRegressor":{
            'depth': [4,6,8,10],
            'l2_leaf_reg': np.logspace(-2,1,10),
            'learning_rate': np.linspace(0.001,0.2,8),
            'iterations': np.arange(10,500,50)
        },
        "AdaBoostRegressor":{
            'learning_rate': np.linspace(0.001,0.2,8),
            'loss':['linear','square','exponential'],
            'n_estimators': np.arange(8,512,50),
        },
        "Lasso":{
            'alpha':  (np.logspace(-8, 8, 100)),
        },
        "ElasticNet": {
            'alpha': np.logspace(-4,2,30),
            'l1_ratio': np.linspace(0,1,30),
            'fit_intercept': [True, False]
        }
 }

In [6]:
def load_dateset(path: str) -> pd.DataFrame:
    """
    Load and preprocess a dataset from a CSV file.

    Removes any rows containing NaN values, and converts certain columns to more memory-efficient data types.
    The columns are then reordered for easier access.

    Parameters:
    path (str): The file path to the CSV file containing the dataset.

    Returns:
    pd.DataFrame
    """
    df = pd.read_csv(path)
    df = df.dropna()
    df["class"] = df["class"].astype(np.int8)
    df["zone_in"] = df["zone_in"].astype(np.int8)
    df["zone_out"] = df["zone_out"].astype(np.int8)
    df["count"] = df["count"].astype(np.int16)
    df["last_15_min_count"] = df["last_15_min_count"].astype(np.int16)
    df["last_30_min_count"] = df["last_30_min_count"].astype(np.int16)
    df = df.iloc[:,[0,1,2,4,5,3]]

    return df


In [7]:
def load_models(model_name: str):
    """
    Initializes and returns a machine learning models. 
    """
    models = {
        "ElasticNet": ElasticNet(copy_X=False),
        "XGBRegressor":XGBRegressor(),
        "LinearRegression":LinearRegression(copy_X=False),
        "RandomForest":RandomForestRegressor(),
        "GradientBoosting":GradientBoostingRegressor(),
        "CatBoostingRegressor":CatBoostRegressor(verbose=False),
        "AdaBoostRegressor":AdaBoostRegressor(),
        "Lasso": Lasso(copy_X=False),
    }

    return models[model_name]

In [8]:
def split_dataset(df: pd.DataFrame, cam_id: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Splits the dataset into matrix of features and dependent variable, scales the input features, and saves the scaler into a pickled file. 

    Parameters:
    -----------
    df (pd.DataFrame): The DataFrame containing the dataset to be split and scaled.
    
    cam_id (str): A unique identifier for the camera or dataset being processed. This ID is used to save the scaler with a specific name.

    Returns:
    --------
    Tuple[np.ndarray, np.ndarray]
    """
    scaler = StandardScaler()
    
    input_feature_train_df = df.drop([target_column], axis=1)
    target_feature_train_df = pd.DataFrame(df[target_column])

    input_feature_train_arr = scaler.fit_transform(input_feature_train_df)

    train_arr = np.c_[
        input_feature_train_arr, np.array(target_feature_train_df)
    ]

    dump(scaler, save_dir / "scalers" / f"{cam_id}.pkl")
    print(f"SC saved to {save_dir / 'scalers' / f'{cam_id}.pkl'}")

    X_train, y_train = train_arr[:, :-1], train_arr[:, -1]

    return X_train, y_train


In [9]:
model_for_location = {
    "18th_Crs_BsStp_JN_FIX_2": "GradientBoosting",
    "Ayyappa_Temple_FIX_1": "CatBoostingRegressor",
    "Stn_HD_1": "CatBoostingRegressor",
    "SBI_Bnk_JN_FIX_3": "CatBoostingRegressor",
    "Ramaiah_BsStp_JN_FIX_2": "CatBoostingRegressor",
    "Mattikere_JN_FIX_1": "GradientBoosting",
    "Buddha_Vihara_Temple": "XGBRegressor",
    "HP_Ptrl_Bnk_BEL_Rd_FIX_2": "CatBoostingRegressor",
    "Sundaranagar_Entrance": "GradientBoosting",
    "80ft_Road": "GradientBoosting",
    "Devasandra_Sgnl_JN_FIX_1": "CatBoostingRegressor",
    "Devasandra_Sgnl_JN_FIX_3": "CatBoostingRegressor",
    "Sty_Wll_Ldge_FIX_3": "GradientBoosting",
    "Ramaiah_BsStp_JN_FIX_1": "CatBoostingRegressor",
    "MS_Ramaiah_JN_FIX_1": "RandomForest",
    "MS_Ramaiah_JN_FIX_2": "CatBoostingRegressor",
    "18th_Crs_Bus_Stop_FIX_2": "CatBoostingRegressor",
    "SBI_Bnk_JN_FIX_1": "CatBoostingRegressor",
    "Mattikere_JN_FIX_3": "CatBoostingRegressor",
    "Mattikere_JN_HD_1": "Lasso",
    "Kuvempu_Circle_FIX_2": "RandomForest",
    "Kuvempu_Circle_FIX_1": "GradientBoosting",
    "Mattikere_JN_FIX_2": "GradientBoosting"
}

### Train models

In [10]:
models_report = dict() # cam_id -> (model_score, model_name, parameters)

# Iterate over each dataset in the dataset directory
for file in os.listdir(dataset_dir):
    if not file.endswith(".csv"): continue

    cam_id = file.removesuffix(".csv")
    print(f"Training for {cam_id} ...")
    models_report[cam_id] = dict()
    
    # Load the dataset
    df = load_dateset(dataset_dir / file)

    # Scaler features and target variable
    X_train, y_train = split_dataset(df, cam_id)

    model_name = model_for_location[cam_id]
    
    # Initialize previous best performing model
    model = load_models(model_name)
    
    os.makedirs(save_dir / "models" / cam_id, exist_ok=True)
    print(f"Now evaluating: {model_name}\n\n")

    para = params[model_name]
    
    # Hyperparameter tuning for GPU supported models
    if model_name in acc_models:
        gs = GridSearchCV(model, para, cv=5, verbose=4, n_jobs=10)
        with parallel_backend('ray', ray_remote_args=dict(num_gpus=0.01)):
            gs.fit(X_train, y_train)
    else:
        # Hyperparameter tuning for non-GPU supported models
        gs = GridSearchCV(model, para, cv=5, verbose=4, n_jobs=-1)
        with parallel_backend('ray'):
            gs.fit(X_train, y_train)

        
    model.set_params(**gs.best_params_)
    # Fit the model with best parameters
    model.fit(X_train, y_train)

    model_score = gs.best_score_
    parameters = gs.best_params_
    
    # Save a pickled copy of the model
    dump(model, save_dir / "models" / cam_id / f"{model_name}.pkl")
    print(f"{model_name} saved to {save_dir / 'models' / cam_id}") 


    models_report[cam_id] = (model_score, model_name, parameters)
    
    print("Best model score and parameters:\n")
    print(models_report[cam_id])
    print("\n\n\n")       

2024-08-25 10:56:03,387	WARNING pool.py:589 -- The 'context' argument is not supported using ray. Please refer to the documentation for how to control ray initialization.


Training for 18th_Crs_BsStp_JN_FIX_2 ...
SC saved to /home/user1/codes/bangaluru_mobility_codes/submission/future_counts/model_training/results_entire_dataset_each_cam_id/scalers/18th_Crs_BsStp_JN_FIX_2.pkl
[I] [10:56:03.213188] Unused keyword parameter: copy_X during cuML estimator initialization
[I] [10:56:03.281393] Unused keyword parameter: copy_X during cuML estimator initialization
Now evaluating: GradientBoosting


Fitting 5 folds for each of 2304 candidates, totalling 11520 fits
GradientBoosting saved to /home/user1/codes/bangaluru_mobility_codes/submission/future_counts/model_training/results_entire_dataset_each_cam_id/models/18th_Crs_BsStp_JN_FIX_2
Best model score and parameters:

(0.9375828231461629, 'GradientBoosting', {'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'huber', 'max_features': 'sqrt', 'n_estimators': 64, 'subsample': 0.8})




Training for Ayyappa_Temple_FIX_1 ...
SC saved to /home/user1/codes/bangaluru_mobility_codes/submission/future_counts/mode

(PoolActor pid=2752347) [CV 2/5] END criterion=squared_error, learning_rate=0.1, loss=squared_error, max_features=sqrt, n_estimators=8, subsample=0.6;, score=0.756 total time=   0.0s
(PoolActor pid=2752428) [CV 1/5] END criterion=squared_error, learning_rate=0.1, loss=squared_error, max_features=log2, n_estimators=8, subsample=0.7;, score=0.766 total time=   0.0s
(PoolActor pid=2752389) [CV 3/5] END criterion=squared_error, learning_rate=0.1, loss=huber, max_features=sqrt, n_estimators=8, subsample=0.8;, score=0.613 total time=   0.0s
(PoolActor pid=2753067) [CV 5/5] END criterion=squared_error, learning_rate=0.1, loss=huber, max_features=log2, n_estimators=8, subsample=0.6;, score=0.547 total time=   0.0s
(PoolActor pid=2753067) [CV 1/5] END criterion=squared_error, learning_rate=0.1, loss=huber, max_features=log2, n_estimators=8, subsample=0.7;, score=0.627 total time=   0.0s
(PoolActor pid=2752382) [CV 1/5] END criterion=squared_error, learning_rate=0.1, loss=absolute_error, max_fea

/home/user1/codes/bangaluru_mobility_codes/base/lib/python3.10/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


CatBoostingRegressor saved to /home/user1/codes/bangaluru_mobility_codes/submission/future_counts/model_training/results_entire_dataset_each_cam_id/models/Ayyappa_Temple_FIX_1
Best model score and parameters:

(0.9539773126983102, 'CatBoostingRegressor', {'depth': 4, 'iterations': 60, 'l2_leaf_reg': 0.01, 'learning_rate': 0.11471428571428573})




Training for Stn_HD_1 ...
SC saved to /home/user1/codes/bangaluru_mobility_codes/submission/future_counts/model_training/results_entire_dataset_each_cam_id/scalers/Stn_HD_1.pkl
[I] [11:11:42.090120] Unused keyword parameter: copy_X during cuML estimator initialization
[I] [11:11:42.090515] Unused keyword parameter: copy_X during cuML estimator initialization
Now evaluating: CatBoostingRegressor


Fitting 5 folds for each of 3200 candidates, totalling 16000 fits
CatBoostingRegressor saved to /home/user1/codes/bangaluru_mobility_codes/submission/future_counts/model_training/results_entire_dataset_each_cam_id/models/Stn_HD_1
Best model score and

...

(PoolActor pid=1415059) [CV 4/5] END criterion=squared_error, learning_rate=0.1, loss=squared_error, max_features=sqrt, n_estimators=8, subsample=0.6;, score=0.779 total time=   0.0s
(PoolActor pid=1413422) [CV 2/5] END bootstrap=True, criterion=poisson, max_features=log2, n_estimators=1024;, score=0.953 total time=   3.5s
(PoolActor pid=1415076) [CV 2/5] END criterion=squared_error, learning_rate=0.1, loss=squared_error, max_features=log2, n_estimators=8, subsample=0.75;, score=0.772 total time=   0.0s
(PoolActor pid=1415083) [CV 1/5] END criterion=squared_error, learning_rate=0.1, loss=huber, max_features=sqrt, n_estimators=8, subsample=0.75;, score=0.725 total time=   0.0s
(PoolActor pid=1415079) [CV 1/5] END criterion=squared_error, learning_rate=0.1, loss=huber, max_features=log2, n_estimators=8, subsample=0.6;, score=0.688 total time=   0.0s
(PoolActor pid=1415079) [CV 2/5] END criterion=squared_error, learning_rate=0.1, loss=huber, max_features=log2, n_estimators=8, subsample=0.

/home/user1/codes/bangaluru_mobility_codes/base/lib/python3.10/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GradientBoosting saved to /home/user1/codes/bangaluru_mobility_codes/submission/future_counts/model_training/results_entire_dataset_each_cam_id/models/Kuvempu_Circle_FIX_1
Best model score and parameters:

(0.955481135448275, 'GradientBoosting', {'criterion': 'squared_error', 'learning_rate': 0.1, 'loss': 'absolute_error', 'max_features': 'sqrt', 'n_estimators': 128, 'subsample': 0.75})




Training for Mattikere_JN_FIX_2 ...
SC saved to /home/user1/codes/bangaluru_mobility_codes/submission/future_counts/model_training/results_entire_dataset_each_cam_id/scalers/Mattikere_JN_FIX_2.pkl
[I] [15:50:43.142823] Unused keyword parameter: copy_X during cuML estimator initialization
[I] [15:50:43.143107] Unused keyword parameter: copy_X during cuML estimator initialization
Now evaluating: GradientBoosting


Fitting 5 folds for each of 2304 candidates, totalling 11520 fits


/home/user1/codes/bangaluru_mobility_codes/base/lib/python3.10/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GradientBoosting saved to /home/user1/codes/bangaluru_mobility_codes/submission/future_counts/model_training/results_entire_dataset_each_cam_id/models/Mattikere_JN_FIX_2
Best model score and parameters:

(0.9745166889694582, 'GradientBoosting', {'criterion': 'squared_error', 'learning_rate': 0.1, 'loss': 'squared_error', 'max_features': 'sqrt', 'n_estimators': 64, 'subsample': 0.7})






In [11]:
models_report

{'18th_Crs_BsStp_JN_FIX_2': (0.9375828231461629,
  'GradientBoosting',
  {'criterion': 'friedman_mse',
   'learning_rate': 0.1,
   'loss': 'huber',
   'max_features': 'sqrt',
   'n_estimators': 64,
   'subsample': 0.8}),
 'Ayyappa_Temple_FIX_1': (0.9539773126983102,
  'CatBoostingRegressor',
  {'depth': 4,
   'iterations': 60,
   'l2_leaf_reg': 0.01,
   'learning_rate': 0.11471428571428573}),
 'Stn_HD_1': (0.9601979753572177,
  'CatBoostingRegressor',
  {'depth': 10,
   'iterations': 160,
   'l2_leaf_reg': 0.09999999999999999,
   'learning_rate': 0.029428571428571432}),
 'SBI_Bnk_JN_FIX_3': (0.9611470844309009,
  'CatBoostingRegressor',
  {'depth': 10,
   'iterations': 160,
   'l2_leaf_reg': 0.09999999999999999,
   'learning_rate': 0.029428571428571432}),
 'Ramaiah_BsStp_JN_FIX_2': (0.9599139671339081,
  'CatBoostingRegressor',
  {'depth': 4,
   'iterations': 210,
   'l2_leaf_reg': 10.0,
   'learning_rate': 0.029428571428571432}),
 'Mattikere_JN_FIX_1': (0.9211983083109135,
  'Gradient